In [1]:
import warnings
from lambeq import QuantumTrainer, SPSAOptimizer
from helper import *
from lambeq import Dataset
import os
import numpy as np
import pandas as pd
import sklearn as sk
#import json

C:\Users\User\.conda\envs\womanium\lib\site-packages\lambeq\text2diagram\ccg_parser.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "true"
BATCH_SIZE = 50
EPOCHS = 120
SEED = 2


In [3]:
df = pd.read_csv("MC1.txt", header=None, sep=", ", engine="python")

In [4]:
df.columns = ["s1", "s2", "label"]

df.head()

,s1,s2,label
0,cook creates complicated dish,experienced chef prepares complicated dish,1
1,skilful programmer creates code,devoted hacker writes code,1
2,skilful cook creates meal,devoted hacker creates complicated code,0
3,hacker writes code,skilful hacker creates code,1
4,devoted hacker writes code,hacker writes complicated code,1


In [5]:
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
df_train_val, df_test = train_test_split(df, test_size=0.2, shuffle=False)
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5)
rskf_splits = list(rskf.split(df_train_val[["s1", "s2"]], y=df_train_val["label"]))

In [7]:
#from discopy.quantum.circuit import Sim14ansatz
from discopy.quantum.gates import CX, Controlled, X
from discopy.tensor import Tensor
from math import pi

In [4]:
pip install --force-reinstall lambeq==0.2.7

  Using cached lambeq-0.2.7-py3-none-any.whl (129 kB)
  Using cached spacy-3.4.4-cp310-cp310-win_amd64.whl (11.9 MB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached torch-1.13.1-cp310-cp310-win_amd64.whl (162.6 MB)
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached discopy-0.4.3-py3-none-any.whl
  Using cached tensornetwork-0.4.6-py3-none-any.whl (364 kB)
  Using cached pytket-1.10.0-cp310-cp310-win_amd64.whl (10.6 MB)
  Using cached Pillow-9.3.0-cp310-cp310-win_amd64.whl (2.5 MB)
  Using cached numpy-1.24.1-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached networkx-2.8.8-py3-none-any.whl (2.0 MB)
  Using cached matplotlib-3.6.2-cp310-cp310-win_amd64.whl (7.2 MB)
  Using cached lark_parser-0.12.0-py2.py3-none-any.whl (103 kB)
  Using cached types_pkg_resources-0.1.3-py2.py3-none-any.whl (4.8 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached scipy-1.9.3-cp310-cp310-win_amd64.whl (40.1 MB)
  Using cach

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\.conda\\envs\\womanium\\Lib\\site-packages\\~il\\_imaging.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [8]:
from lambeq import  AtomicType
#from lambeq.ansatz.circuit import _ArMapT
from lambeq.ansatz.circuit import Sim15Ansatz
# Atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

ansatz = Sim15Ansatz({N: 3, S: 1}, n_layers=2)

ImportError: cannot import name 'Sim15Ansatz' from 'lambeq.ansatz.circuit' (C:\Users\User\.conda\envs\womanium\lib\site-packages\lambeq\ansatz\circuit.py)

In [ ]:
train_idx, val_idx = rskf_splits[0]
df_train, df_val = df_train_val.iloc[train_idx], df_train_val.iloc[val_idx]
df_train.shape

In [ ]:
print(df_train[df_train["label"] == 0].shape)
print(df_train[df_train["label"] == 1].shape)

In [ ]:
parser = BobcatParser(verbose='text')
preprocess_df(df_train, ansatz)
preprocess_df(df_val, ansatz)
preprocess_df(df_test, ansatz)
df.dropna()


In [ ]:
train_circuits = list(df_train[["s1_circuit", "s2_circuit"]].values)
val_circuits = list(df_val[["s1_circuit", "s2_circuit"]].values)
test_circuits = list(df_test[["s1_circuit", "s2_circuit"]].values)
print(train_circuits[0][0].draw())


In [ ]:
all_circuits = train_circuits + val_circuits + test_circuits
assert len(all_circuits) == len(df)
train_diagrams = list(df_train[["s1_diagrams", "s2_diagrams"]].values)
val_diagrams = list(df_val[["s1_diagrams", "s2_diagrams"]].values)
test_diagrams = list(df_test[["s1_diagrams", "s2_diagrams"]].values)
all_diagrams = train_diagrams + val_diagrams + test_diagrams         
np.array(all_circuits).reshape(-1).shape

In [ ]:
train_labels = list(df_train["label_v"].values)
val_labels = list(df_val["label_v"].values)
test_labels = list(df_test["label_v"].values)
assert len(train_labels) > 0
assert(test_labels)
assert(val_labels)

In [ ]:
from pytket.extensions.qiskit import IBMQBackend
from pytket.extensions.qiskit import AerBackend
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ
IBMQ.save_account('3c7bb882c186256cc5e989cf02f639e9282a264ee520c8882c6fb1c9ea51be3ce3d407629b8fed7450de430b7ccc59d4f97afddbba9b72b9daaebb40559ef0fc')
provider = IBMQ.load_account()
backend = AerBackend(NoiseModel.from_backend(IBMQ.providers()[0].get_backend('ibmq_manila')))

backend_config = {
    "backend": backend,
    "compilation": backend.default_compilation_pass(2),
    "shots": 2**10,
}

In [ ]:
model = CustomTketModel.from_diagrams(np.array(all_circuits).reshape(-1), backend_config=backend_config)

In [ ]:
epsilon=np.finfo('float').eps
loss = lambda y_hat, y: (-np.nansum(y * np.log(y_hat + epsilon)) + epsilon ) / len(y)
acc = lambda y_hat, y: np.nansum(np.round(y_hat) == y) / len(y)/ 2 if (len(y) > 0) else None
eval_metrics = {"acc": acc}

In [ ]:
trainer = QuantumTrainer(
    model,
    loss_function=loss,
    epochs=EPOCHS,
    optimizer=SPSAOptimizer,
    optim_hyperparams={"a": 0.05, "c": 0.06, "A": 0.01 * EPOCHS},
    evaluate_functions=eval_metrics,
    verbose="text",
    seed=SEED,
)

In [ ]:

train_dataset = Dataset(
            train_circuits,
            train_labels,
            batch_size=BATCH_SIZE)

val_dataset = Dataset(val_circuits, val_labels, shuffle=False)
if not train_labels:
    raise Exception("Empty train labels list!")
if not val_labels:
    raise Exception("Empty test labels list!")
print(val_labels)

In [ ]:
import time
start_time = time.time()
trainer.fit(train_dataset, val_dataset, logging_step=12)

print("Training Time:--- %s seconds ---" % (time.time() - start_time))


In [ ]:
np.save('accuracy_vals_train_Sim15_N3_l2_noise2.npy', trainer.train_results['acc'])
np.save('accuracy_vals_val_Sim15_N3_l2_noise2.npy', trainer.val_results['acc'])
np.save('cost_vals_train_Sim15_N3_l2_noise2.npy', trainer.train_epoch_costs)
np.save('cost_vals_val_Sim15_N3_l2_noise2.npy', trainer.val_costs)

In [ ]:
import matplotlib.pyplot as plt

fig1, ((ax_tl, ax_tr), (ax_bl, ax_br)) = plt.subplots(2, 2, sharey='row', figsize=(10, 6))

ax_tl.set_title('Training set')
ax_tr.set_title('Development set')
ax_bl.set_xlabel('Epochs')
ax_br.set_xlabel('Epochs')
ax_bl.set_ylabel('Accuracy')
ax_tl.set_ylabel('Loss')

colours = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
ax_tl.plot(trainer.train_epoch_costs, color=next(colours))

ax_bl.plot(trainer.train_results['acc'], color=next(colours))

ax_tr.plot(trainer.val_costs, color=next(colours))

ax_br.plot(trainer.val_results['acc'], color=next(colours))


# print test accuracy
test_acc = acc(model(test_circuits), test_labels)
print('Test accuracy:', test_acc.item())

In [ ]:
from discopy.quantum.circuit import IQPansatz as IQP
pprint = lambda c: print(str(c).replace(' >>', '\n  >>'))
pprint(IQP(2, [[0.1], [0.3]]))

In [ ]:

from collections.abc import Mapping
from lambeq import CircuitAnsatz
from typing import Any, Callable, Optional

from discopy.quantum.circuit import (Circuit, Discard, Functor, Id,
                                     IQPansatz as IQP, qubit, Sim14ansatz)
from discopy.quantum.gates import Bra, Ket, Rx, Rz
from discopy.rigid import Box, Diagram, Ty

from lambeq.ansatz import Symbol
from lambeq.ansatz.circuit import _ArMapT


class CustomAnsatz(CircuitAnsatz):
    """S YC Chen, et al's deep reinforcement learning ansatz."""

    def __init__(self,
                 ob_map: Mapping[Ty, int],
                 n_layers: int,
                 n_single_qubit_params: int = 3,
                 discard: bool = False,
                 special_cases: Optional[Callable[[_ArMapT], _ArMapT]] = None):
        """

        Parameters
        ----------
        ob_map : dict
            A mapping from :py:class:`discopy.rigid.Ty` to the number of
            qubits it uses in a circuit.
        n_layers : int
            The number of IQP layers used by the ansatz.
        n_single_qubit_params : int, default: 3
            The number of single qubit rotations used by the ansatz.
        discard : bool, default: False
            Discard open wires instead of post-selecting.
        special_cases : callable, optional
            A function that transforms an arrow map into one specifying
            special cases that should not be converted by the Ansatz
            class.

        """
        super().__init__(ob_map=ob_map, n_layers=n_layers,
                         n_single_qubit_params=n_single_qubit_params)

        if special_cases is None:
            special_cases = self._special_cases

        self.n_layers = n_layers
        self.n_single_qubit_params = n_single_qubit_params
        self.discard = discard
        self.functor = Functor(ob=self.ob_map,
                               ar=special_cases(self._ar))

    def _ar(self, box: Box) -> Circuit:
        label = self._summarise_box(box)
        dom, cod = self._ob(box.dom), self._ob(box.cod)

        n_qubits = max(dom, cod)
        n_layers = self.n_layers
        n_1qubit_params = self.n_single_qubit_params

        if n_qubits == 0:
            circuit = Id()
        elif n_qubits == 1:
            syms = [Symbol(f'{label}_{i}') for i in range(n_1qubit_params)]
            rots = [Rx, Rz]
            circuit = Id(qubit)
            for i, sym in enumerate(syms):
                circuit >>= rots[i % 2](sym)
        else:
            # We have three rotations for each qubit corresponding to
            # the Euler decomposition
            n_params = n_layers * n_qubits * 4
            syms = [Symbol(f'{label}_{i}') for i in range(n_params)]
            params: np.ndarray[Any, np.dtype[Any]] = np.array(syms).reshape(
                    (n_layers, n_qubits, 3))
            circuit = Sim14ansatz(n_qubits, params)

        if cod > dom:
            circuit <<= Id(dom) @ Ket(*[0]*(cod - dom))
        elif self.discard:
            circuit >>= Id(cod) @ Discard(dom - cod)
        else:
            circuit >>= Id(cod) @ Bra(*[0]*(dom - cod))
        return circuit

In [ ]:
from lambeq import BobcatParser, remove_cups
params = [[0.1, 0.2, 0.3, 0.3, 0.4, 0.5, 0.5, 0.6, 0.7, 0.8, 0.9, 0.9],
          [0.1, 0.2, 0.3, 0.3, 0.4, 0.5, 0.5, 0.6, 0.7, 0.8, 0.9, 0.9],
         [0.1, 0.2, 0.3, 0.3, 0.4, 0.5, 0.5, 0.6, 0.7, 0.8, 0.9, 0.9]]
sentence = "John cooks delicious food"
parser = BobcatParser()
diagram = parser.sentence2diagram(sentence)

In [ ]:
# Atomic types
from lambeq import remove_cups, AtomicType
import numpy as np
N = AtomicType.NOUN
S = AtomicType.SENTENCE
Sim14Ansatz = _sim_ansatz_factory(14)
Sim15Ansatz = _sim_ansatz_factory(15)
ansatz = Sim14Ansatz({N: 1, S: 1}, n_layers=1)
discopy_Circ2 = ansatz(remove_cups(diagram))
discopy_Circ2.draw(figsize=(20, 20), fontsize=12)
discopy_circuit = Sim14ansatz(3, params)
discopy_circuit.draw(figsize=(20, 20), fontsize=12)

In [ ]:

from abc import ABC, abstractmethod
from collections.abc import Callable, Mapping
from datetime import datetime
from math import ceil
import os
import random
import socket
import sys
from typing import Any, Optional, Union
from typing import TYPE_CHECKING

from discopy import Tensor
from tqdm.notebook import tqdm, trange

if TYPE_CHECKING:
    from torch.utils.tensorboard import SummaryWriter

from lambeq.core.globals import VerbosityLevel
from lambeq.training.checkpoint import Checkpoint
from lambeq.training.dataset import Dataset
from lambeq.training.model import Model


def _import_tensorboard_writer() -> None:
    global SummaryWriter
    try:
        from torch.utils.tensorboard import SummaryWriter
    except ImportError:  # pragma: no cover
        raise ImportError('tensorboard not found. Please install it using '
                          '`pip install tensorboard`.')


_StrPathT = Union[str, 'os.PathLike[str]']


class CustomQuantumTrainer(QuantumTrainer):


    def validation_step(
            self,
            batch: tuple[list[Any], np.ndarray]) -> tuple[np.ndarray, float]:
        """Perform a validation step.
        Parameters
        ----------
        batch : tuple of list and np.ndarray
            Current batch.
        Returns
        -------
        tuple of np.ndarray and float
            The model predictions and the calculated loss.
        """
        x, y = batch
        y_hat = self.model(x)
        assert len(y) > 0
        #assert y_hat != 0
        print('Size of y from validation function', len(y))
        print('log(y_hat) from validation function', np.log(y_hat))
        loss = self.loss_function(y_hat, y)
        numerator_loss = -np.sum(y * np.log(y_hat + epsilon)) + epsilon
        denom_loss = len(y)
        print('calculated numerator loss from validation function', numerator_loss)
        print('calculated denom loss from validation function', denom_loss)
        print('calculated loss from validation function', loss)
        return y_hat, loss
    def fit(self,
            train_dataset: Dataset,
            val_dataset: Optional[Dataset] = None,
            evaluation_step: int = 1,
            logging_step: int = 1) -> None:
        """Fit the model on the training data and, optionally,
        evaluate it on the validation data.

        Parameters
        ----------
        train_dataset : :py:class:`Dataset`
            Dataset used for training.
        val_dataset : :py:class:`Dataset`, optional
            Validation dataset.
        evaluation_step : int, default: 1
            Sets the intervals at which the metrics are evaluated on the
            validation dataset.
        logging_step : int, default: 1
            Sets the intervals at which the training statistics are
            printed if `verbose = 'text'` (otherwise ignored).

        """
        if self.from_checkpoint:
            self._load_extra_chkpoint_info(self.checkpoint)

        def writer_helper(*args: Any) -> None:
            if self.use_tensorboard:
                self.writer.add_scalar(*args)
            else:
                print(*args)

        # initialise progress bar
        step = self.start_step
        batches_per_epoch = ceil(len(train_dataset)/train_dataset.batch_size)
        status_bar = tqdm(total=float('inf'),
                          bar_format='{desc}',
                          desc=self._generate_stat_report(),
                          disable=(
                                self.verbose != VerbosityLevel.PROGRESS.value),
                          leave=True,
                          position=0)

        # start training loop
        for epoch in trange(self.start_epoch,
                            self.epochs,
                            desc='Epoch',
                            disable=(
                                self.verbose != VerbosityLevel.PROGRESS.value),
                            leave=False,
                            position=1):
            train_loss = 0.0
            with Tensor.backend(self.backend):
                for batch in tqdm(train_dataset,
                                  desc='Batch',
                                  total=batches_per_epoch,
                                  disable=(self.verbose
                                           != VerbosityLevel.PROGRESS.value),
                                  leave=False,
                                  position=2):
                    step += 1
                    x, y_label = batch
                    y_hat, loss = self.training_step(batch)
                    writer_helper('y_hat', y_hat)
                    #if np.isnan(np.array(y_hat)).any():
                     #   raise Exception("Empty evaluation values list after training step!")
                    #if np.isnan(np.array(loss)).any():
                     #   raise Exception("Nan loss values list after training step!")
                    writer_helper('y_hat', y_hat)
                    if np.isnan(np.array(y_hat)).any():
                        raise Exception("Empty evaluation values list!")
                    if (self.evaluate_on_train
                            and self.evaluate_functions is not None):
                        for metr, func in self.evaluate_functions.items():
                            res = func(y_hat, y_label)
                            metric = self._train_results_epoch[metr]
                            metric.append(len(x) * res)
                    train_loss += len(batch[0]) * loss
                    assert np.isnan(np.array(loss)).any() == False
                    writer_helper('train/step_loss', loss, step)
                    status_bar.set_description(
                            self._generate_stat_report(
                                train_loss=loss,
                                val_loss=(self.val_costs[-1] if self.val_costs
                                          else None)))
            train_loss /= len(train_dataset)
            assert len(train_dataset) > 0
            print('y_label', y_label)
            if np.isnan(np.array(y_label)).any():
                raise Exception("Empty train label list!")
            if np.isnan(np.array(y_hat)).any():
                raise Exception("Empty evaluation values list!")
            self.train_epoch_costs.append(train_loss)
            writer_helper('train/epoch_loss', train_loss, epoch + 1)

            # evaluate on train
            if (self.evaluate_on_train
                    and self.evaluate_functions is not None):
                for name in self._train_results_epoch:
                    self.train_results[name].append(
                        sum(self._train_results_epoch[name])/len(train_dataset)
                    )
                    self._train_results_epoch[name] = []  # reset
                    writer_helper(
                        f'train/{name}', self.train_results[name][-1],
                        epoch+1)
                    if self.verbose == VerbosityLevel.PROGRESS.value:
                        status_bar.set_description(
                                self._generate_stat_report(
                                    train_loss=train_loss,
                                    val_loss=(self.val_costs[-1]
                                              if self.val_costs else None)))

            # evaluate metrics on validation data
            if val_dataset is not None:
                if epoch % evaluation_step == 0:
                    val_loss = 0.0
                    batches_per_validation = ceil(len(val_dataset)
                                                  / val_dataset.batch_size)
                    writer_helper('batches_per_validation', batches_per_validation, len(val_dataset), val_dataset.batch_size)
                    with Tensor.backend(self.backend):
                        disable_tqdm = (self.verbose
                                        != VerbosityLevel.PROGRESS.value)
                        for v_batch in tqdm(val_dataset,
                                            desc='Validation batch',
                                            total=batches_per_validation,
                                            disable=disable_tqdm,
                                            leave=False,
                                            position=2):
                            writer_helper("***", v_batch)
                            x_val, y_label_val = v_batch
                            writer_helper("***", x_val, y_label_val)
                            y_hat_val, cur_loss = self.validation_step(v_batch)
                            if np.isnan(np.array(y_hat)).any():
                                raise Exception("Empty evaluation values list after validation step!")
                            writer_helper("***", y_hat_val, cur_loss)
                            val_loss += cur_loss * len(x_val)
                            if self.evaluate_functions is not None:
                                for metr, func in (
                                        self.evaluate_functions.items()):
                                    res = func(y_hat_val, y_label_val)
                                    self._val_results_epoch[metr].append(
                                        len(x_val)*res)
                            status_bar.set_description(
                                    self._generate_stat_report(
                                        train_loss=train_loss,
                                        val_loss=val_loss))
                        assert len(val_dataset) > 0
                        writer_helper('empty val dataset!', len(val_dataset))
                        val_loss /= len(val_dataset)
                        self.val_costs.append(val_loss)
                        writer_helper('cost vals', self.val_costs)
                        status_bar.set_description(
                                self._generate_stat_report(
                                    train_loss=train_loss,
                                    val_loss=val_loss))
                        if np.isnan(np.array(val_loss)).any():
                            raise Exception("loss value is NaN!")
                        writer_helper('empty val dataset!', len(val_dataset))
                        writer_helper('val/loss', val_loss, epoch+1)

                    if self.evaluate_functions is not None:
                        for name in self._val_results_epoch:
                            assert len(val_dataset) > 0
                            writer_helper('empty val dataset!', len(val_dataset))
                            self.val_results[name].append(
                                sum(self._val_results_epoch[name])
                                / len(val_dataset))
                            self._val_results_epoch[name] = []  # reset
                            writer_helper(
                                f'val/{name}', self.val_results[name][-1],
                                epoch + 1)
                            status_bar.set_description(
                                    self._generate_stat_report(
                                        train_loss=train_loss,
                                        val_loss=val_loss))
            # save checkpoint info
            save_dict = {'epoch': epoch+1,
                         'model_weights': self.model.weights,
                         'model_symbols': self.model.symbols,
                        'train_costs': self.train_costs,
                         'train_epoch_costs': self.train_epoch_costs,
                         'train_results': self.train_results,
                         'val_costs': self.val_costs,
                         'val_results': self.val_results,
                         'random_state': random.getstate(),
                         'step': step}
            print(f"save_dict: {save_dict}")
            self.save_checkpoint(save_dict, self.log_dir)
            if self.verbose == VerbosityLevel.TEXT.value:
                if epoch == 0 or (epoch+1) % logging_step == 0:
                    space = (len(str(self.epochs))-len(str(epoch+1)) + 2) * ' '
                    prefix = f'Epoch {epoch+1}:' + space
                    print(prefix + self._generate_stat_report(
                            train_loss=train_loss,
                            val_loss=(self.val_costs[-1] if self.val_costs
                                      else None)),
                          file=sys.stderr)
        status_bar.close()
        if self.verbose == VerbosityLevel.TEXT.value:
            print('\nTraining completed!', file=sys.stderr)

In [ ]:
trainer2 = CustomQuantumTrainer(
    model,
    epochs=EPOCHS,
    loss_function=loss,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.05, 'c': 0.06, 'A':0.01*EPOCHS},
    evaluate_functions={'acc': acc},
    evaluate_on_train=True,
    verbose = 'text',
    seed=0
)

In [ ]:
trainer2.fit(train_dataset, val_dataset, logging_step=12)

In [ ]:
from discopy.quantum import Circuit

In [ ]:
np.array(all_circuits).reshape(-1)
all_circuits[0]

In [ ]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(all_circuits[0][0].to_tk())

In [ ]:
lambdified_diagram = model._get_lambda(train_circuits[0][0])
print(len(val_circuits))

In [ ]:
print(lambdified_diagram)

In [ ]:
print(len(model.weights))


In [ ]:
tensors = Circuit.eval(
    *lambdified_diagram(*model.weights),
    **model.backend_config,
    seed=model._randint(),
)

In [ ]:
lambdified_diagram(*model.weights)

In [ ]:
import numpy as np
epsilon=np.finfo('float64').eps
eps = 1e-50
np.log(eps + epsilon)/16

In [ ]:
import numpy as np
np.log(0)